# Jupyter Notebook - Ransomware report use cases 2

Copyright © 2021 Google

In [1]:
import vt
import nest_asyncio
import pandas as pd

In [ ]:
#@markdown Please, insert your VT API Key*:

API_KEY = ''  #@param  {type: "string"}

#@markdown **The API key should have Premium permissions, otherwise some of the use cases might not provide the expected results.*

#@markdown 

In [2]:
nest_asyncio.apply()

# 1. Collecting hashes

In [23]:
nh = 0
with vt.Client(API_KEY) as client:
    it = client.iterator('/intelligence/search',
        params={'query': 'engines:ryuk  fs:2021-01-01+ (type:peexe or type:pedll) p:10+'})
    with open('hashes.log','w') as f:
        for obj in it:
            if obj.id:
                f.write(f'{obj.id}\n')
                nh += 1
    f.close()
print(f'{nh} hashes have been written to the file hashes.log\n')

9426 hashes have been written to the file hashes.log



# 2. Malicious contacted IOCs

In [24]:
def get_malicious_IPs(file_hash):
    contacted_IPs = set()
    for ip in client.iterator('/files/{}/contacted_ips', file_hash, limit=20):
        stats = ip.get('last_analysis_stats')
        if stats and stats['malicious'] >= min_positives:
            contacted_IPs.add(ip.id)
    return contacted_IPs

def get_malicious_Domains(file_hash):
    contacted_domains = set()
    for domain in client.iterator('/files/{}/contacted_domains', file_hash, limit=20):
        stats = domain.get('last_analysis_stats')
        if stats and stats['malicious'] >= min_positives:
            contacted_domains.add(domain.id)
    return contacted_domains

In [25]:
malIOCs = set()
min_positives = 5 # Minimun number of positives for every IOC
nIOCs = 0

with vt.Client(API_KEY) as client:
    it = client.iterator('/intelligence/search',
        params={'query': 'engines:babuk fs:2021-07-01+ (have:contacted_domains or have:contacted_ips)'})
    for obj in it:
        ips = get_malicious_IPs(obj.id)
        domains = get_malicious_Domains(obj.id)
        malIOCs = malIOCs | ips | domains

with open('iocs.log','w') as f:
    for ioc in malIOCs:
        f.write(f'{ioc}\n')
        nIOCs += 1
        
print(f'{nIOCs} IOCs have been written to the file iocs.log\n')        
f.close()

29 IOCs have been written to the file iocs.log



# 3. Searching for a text in domains and URLs

In [3]:
def search_domains(file_hash, text):
    domains = set()
    for domain in client.iterator('/files/{}/embedded_domains', file_hash, limit=20):
        if text in domain.id:
            domains.add(domain.id)
    for domain in client.iterator('/files/{}/itw_domains', file_hash, limit=20):
        if text in domain.id:
            domains.add(domain.id)            
    return domains

def search_urls(file_hash, text):
    urls = set()
    for url in client.iterator('/files/{}/embedded_urls', file_hash, limit=20):
        if text in url.id:
            urls.add(url.id)
    for url in client.iterator('/files/{}/itw_urls', file_hash, limit=20):
        if text in url.id:
            urls.add(url.id)
    return urls

In [13]:
stxt="google" # Write here any text you want to search for
query_string = 'engines:cerber fs:2021-06-01+ (embedded_domains: %s OR embedded_urls:%s OR itw: %s)' % (stxt, stxt, stxt)

with vt.Client(API_KEY) as client:
    it = client.iterator('/intelligence/search', params={'query': query_string})

    for obj in it:
        results = search_domains(obj.id, search_text) | search_urls(obj.id, search_text)
        
        if results:
            print(f'** Results found for file hash: {obj.id} (%s)\n' %(obj.type_tag) )
            for result in results:
                print(f' - {result}\n')

** Results found for file hash: 1a5f649438646bfe909ac1e08bfe37f969e21b7900933d51ad2fe0c5c549a79b (pedll)

 - www.google.fr

** Results found for file hash: af48fb4200cff418d513fe0ca841a8adb8699b266266fcf46ef44e460361cff0 (peexe)

 - www.google-analytics.com

** Results found for file hash: 062227eaca57abaf97dd2c6d5f198fdd0d11b15cfd45eef0f0f0217d5c70aa06 (android)

 - play.google.com

 - googleads.g.doubleclick.net

 - pagead2.googlesyndication.com

 - plus.google.com

 - www.google.com

 - support.google.com

** Results found for file hash: 569a267a4282bc694a8b39502b4e51bd3b78d449e075f4480c51eba71433b5dd (peexe)

 - www.google.de

** Results found for file hash: 87ceec489fbfc70c2d935dc0717dba6f7d7147d06be08c661e05f7c83c3b67f0 (android)

 - play.google.com

 - googleads.g.doubleclick.net

 - pagead2.googlesyndication.com

 - plus.google.com

 - www.google.com

 - support.google.com

** Results found for file hash: d419b2e7a6e9252b1cb3b86a6f8f107206c31cc666a74af97738f46b650ed4c5 (peexe)
